# Final worksheet to read data from csv, fill missing data, estimate income, cluster the users, find similarities between products, recommend products based on similarity and cluster

**Import all the required libraries, create required functions: func1 and func2, read data from csv**

In [90]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display

def func1(data11):
    if(np.isnan(data11.Age)):
        gender = data11.Gender
        purchase = data11.purchase
        payment = data11['Method of Payment']
        data11.Age = MedianAge[gender][payment][purchase]
    return data11

def func2(data11):
    if(np.isnan(data11['Amount Spent on Goods'])):
        gender = data11.Gender
        purchase = data11.purchase
        payment = data11['Method of Payment']
        data11['Amount Spent on Goods'] = MedianAmount[gender][payment][purchase]
    return data11
    
    


data = pd.read_csv('dataset.csv')

data.info()
df = data.copy()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
Unnamed: 0                                   1000 non-null int64
Retail Purchases – Most Frequent Category    1000 non-null object
Gender                                       1000 non-null object
Method of Payment                            1000 non-null object
Address                                      1000 non-null object
Age                                          826 non-null float64
Estimated Income                             789 non-null float64
Amount Spent on Goods                        826 non-null float64
dtypes: float64(3), int64(1), object(4)
memory usage: 62.6+ KB


**Extracting location information**

In [91]:
location_df = df.Address.apply(lambda x: pd.Series(x.split(' ')))
location_df['Street Address'] = location_df[1] +' '+ location_df[2]
location_df['Ville Address'] = location_df[4] +' '+ location_df[5]
location_df['Pin'] = location_df[3]
location_df= location_df.drop([0,1,2,3,4,5], axis =1)
dataup = df.join(location_df, how='outer')

**Applied few column manipulations**

In [92]:

dataup['purchase'] = dataup['Retail Purchases – Most Frequent Category']
dataup['Retail Purchases – Most Frequent Category'] = dataup['Retail Purchases \xe2\x80\x93 Most Frequent Category']
dataup.drop(['Retail Purchases – Most Frequent Category'], axis =1, inplace=True)



**Fill missing values in Age and Amount spent on goods**

In [93]:
g1 = dataup.sort_values(by='Age').groupby(['Gender','Method of Payment','purchase'])
MedianAge = g1['Age'].median().round()

newdata = dataup.copy()
newdata['number']=newdata['Unnamed: 0']
newdata.drop(['Unnamed: 0','Address'], axis= 1, inplace=True)
newdata.fillna(np.NaN, inplace=True)


updated = newdata.apply(func1, axis=1)
income = updated['Estimated Income']
Amount = updated['Amount Spent on Goods']
updated.drop(['Estimated Income', 'Amount Spent on Goods'], inplace=True, axis =1)
g2 = updated.sort_values(by='Age').groupby(['Gender','Method of Payment','purchase'])
MedianAmount = g1['Amount Spent on Goods'].median()
updated = newdata.apply(func1, axis=1)

amountmissing = updated[updated['Amount Spent on Goods'].isnull()]
incomemissing = updated[updated['Estimated Income'].isnull()].copy()

updated2 = updated.apply(func2, axis=1)

**Estimating the income and storing the updated value in *updated3* variable**

In [94]:
trainingdata = updated2.copy()
trainingdata.dropna(inplace=True)
trainingdata.drop('Street Address', inplace=True, axis =1)
trainingdata.drop('Pin', inplace=True, axis =1)
trainingdata.drop('number', inplace=True, axis =1)
incomesmiss = trainingdata['Estimated Income'].copy()
trainingdata1 = trainingdata.copy()
trainingdata1.drop(['Estimated Income'], axis =1, inplace=True)
incomemissing = updated2[updated2['Estimated Income'].isnull()].copy()
incomemissingcopy = incomemissing.copy()
incomemissingcopy.drop('number', inplace=True, axis =1)
incomemissingcopy.drop('Pin', inplace=True, axis =1)
incomemissingcopy.drop('Street Address', inplace=True, axis =1)

incomemissingcopy.drop('Estimated Income', inplace=True, axis =1)
incomemissingcopy['Gender'] = incomemissingcopy.Gender.astype('category')
incomemissingcopy['Method of Payment'] = incomemissingcopy['Method of Payment'].astype('category')
incomemissingcopy['Ville Address'] = incomemissingcopy['Ville Address'].astype('category')
incomemissingcopy['purchase'] = incomemissingcopy['purchase'].astype('category')

trainingdata1.Gender = le1.transform(trainingdata1.Gender)
trainingdata1.purchase = le2.transform(trainingdata1.purchase)
trainingdata1['Method of Payment'] = le3.transform(trainingdata1['Method of Payment'])
trainingdata1['Ville Address'] = le4.transform(trainingdata1['Ville Address'])

incomemissingcopy.Gender = le1.transform(incomemissingcopy.Gender)
incomemissingcopy.purchase = le2.transform(incomemissingcopy.purchase)
incomemissingcopy['Method of Payment'] = le3.transform(incomemissingcopy['Method of Payment'])
incomemissingcopy['Ville Address'] = le4.transform(incomemissingcopy['Ville Address'])

X_train, X_test, y_train, y_test = train_test_split(trainingdata1, incomesmiss, test_size=0.4, random_state=42)

clf = LinearRegression()
clf.fit(X_train, y_train)
print(clf.score(X_test,y_test))

incomemissingcopyvalues = clf.predict(incomemissingcopy)


incomemissing['Estimated Income'] = incomemissingcopyvalues

updated3 = updated2.copy()
for value in incomemissing.number:
    updated3.loc[updated3['number'] == value] = incomemissing.loc[incomemissing['number'] == value]




0.520494430376


# Clustering

In [101]:
clusterdata = updated3.copy()
clusterdata.Gender = le1.transform(clusterdata.Gender)
clusterdata.purchase = le2.transform(clusterdata.purchase)
clusterdata['Method of Payment'] = le3.transform(clusterdata['Method of Payment'])
le4.fit(clusterdata['Ville Address'])
clusterdata['Ville Address'] = le4.transform(clusterdata['Ville Address'])
le5 = LabelEncoder()
le5.fit(clusterdata['Street Address'])
clusterdata['Street Address'] = le5.transform(clusterdata['Street Address'])

clusterer = KMeans(n_clusters=2, random_state=29).fit(clusterdata)

preds = clusterer.predict(clusterdata)
centers = clusterer.cluster_centers_
score = silhouette_score(clusterdata, clusterer.labels_, metric='euclidean')
print("KMeans score", score)

segments = ['Segment {}'.format(i) for i in range(0,len(centers))]
true_centers = pd.DataFrame(np.round(centers), columns = clusterdata.keys())
true_centers.index = segments
display(true_centers)

clusterdata.Gender = le1.inverse_transform(clusterdata.Gender)
clusterdata.purchase = le2.inverse_transform(clusterdata.purchase)

clusterdata['Method of Payment'] = le3.inverse_transform(clusterdata['Method of Payment'])
clusterdata['Ville Address'] = le4.inverse_transform(clusterdata['Ville Address'])
clusterdata['Street Address'] = le5.inverse_transform(clusterdata['Street Address'])


group1 = pd.DataFrame(columns=clusterdata.columns)
group2 = pd.DataFrame(columns=clusterdata.columns)
for i in range(0,1000):
    if(preds[i]):
        group1 = group1.append(clusterdata.loc[i])
    else:
        group2 = group2.append(clusterdata.loc[i])
        


('KMeans score', 0.50874056595535633)


,Gender,Method of Payment,Age,Estimated Income,Amount Spent on Goods,Street Address,Ville Address,Pin,purchase,number
Segment 0,1.0,0.0,37.0,32944.0,22.0,266.0,180.0,75431.0,3.0,497.0
Segment 1,0.0,0.0,38.0,34868.0,24.0,253.0,187.0,25958.0,3.0,502.0


# Recommendation system

In [103]:
productlist1 = group1.copy()
productlist2 = group2.copy()
productlist1.drop(['Street Address', 'Pin', 'number', 'Estimated Income'], axis =1, inplace=True)
productlist2.drop(['Street Address', 'Pin', 'number', 'Estimated Income'], axis =1, inplace=True)

productlist1.Gender = le1.transform(productlist1.Gender)
productlist1.purchase = le2.transform(productlist1.purchase)

productlist1['Method of Payment'] = le3.transform(productlist1['Method of Payment'])
productlist1['Ville Address'] = le4.transform(productlist1['Ville Address'])

cs= cosine_similarity(productlist1)

pds = pd.DataFrame(cs)
pds.iloc[1].nlargest(n=3, keep = 'first').index[1]

495

# Dont refer below

In [50]:
updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
Gender                   1000 non-null object
Method of Payment        1000 non-null object
Age                      1000 non-null float64
Estimated Income         789 non-null float64
Amount Spent on Goods    826 non-null float64
Street Address           1000 non-null object
Ville Address            1000 non-null object
Pin                      1000 non-null object
purchase                 1000 non-null object
number                   1000 non-null int64
dtypes: float64(3), int64(1), object(6)
memory usage: 78.2+ KB


In [25]:


X_train, X_test, y_train, y_test = train_test_split(trainingdata1, incomesmiss, test_size=0.4, random_state=42)

clf = LinearRegression()
clf.fit(X_train, y_train)
clf.score(X_test,y_test)
incomemissingcopyvalues = clf.predict(trainingdata1)
incomemissing['Estimated Income'] = incomemissingcopyvalues
incomemissingcopy = incomemissing.copy()


ValueError: Length of values does not match length of index

In [85]:
incomemissingcopyvalues.size

789

In [100]:
group1.head()

,Gender,Method of Payment,Age,Estimated Income,Amount Spent on Goods,Street Address,Ville Address,Pin,purchase,number
0,1.0,M-Commerce,42.0,35463.227720,25.299071,Roach Street,Bradley Ville,46040,Jeans,0.0
2,1.0,E-Commerce,45.0,53469.000000,52.520942,Mckinney Street,John Ville,22343,Shirt,2.0
3,1.0,E-Commerce,25.0,20844.000000,24.962752,Smith Street,Noah Ville,42681,Shorts,3.0
4,1.0,E-Commerce,34.0,31656.768028,19.829491,Wheeler Street,Richard Ville,49042,Shorts,4.0
6,1.0,In Store,20.0,17607.000000,3.273650,Edwards Street,Brandon Ville,25601,Jeans,6.0


In [42]:
trainingdata1.Gender

1      LabelEncoder()
2      LabelEncoder()
3      LabelEncoder()
5      LabelEncoder()
6      LabelEncoder()
7      LabelEncoder()
8      LabelEncoder()
10     LabelEncoder()
11     LabelEncoder()
12     LabelEncoder()
14     LabelEncoder()
15     LabelEncoder()
16     LabelEncoder()
17     LabelEncoder()
18     LabelEncoder()
19     LabelEncoder()
21     LabelEncoder()
22     LabelEncoder()
23     LabelEncoder()
24     LabelEncoder()
25     LabelEncoder()
26     LabelEncoder()
27     LabelEncoder()
28     LabelEncoder()
30     LabelEncoder()
31     LabelEncoder()
33     LabelEncoder()
34     LabelEncoder()
35     LabelEncoder()
36     LabelEncoder()
            ...      
962    LabelEncoder()
963    LabelEncoder()
966    LabelEncoder()
967    LabelEncoder()
968    LabelEncoder()
969    LabelEncoder()
970    LabelEncoder()
971    LabelEncoder()
972    LabelEncoder()
974    LabelEncoder()
976    LabelEncoder()
977    LabelEncoder()
979    LabelEncoder()
980    LabelEncoder()
983    Lab

In [33]:
le1.classes_

array(['Female', 'Male'], dtype=object)

In [37]:
le1.transform(['Female'])

array([0])

In [57]:


trainingdata = updated.copy()
trainingdata.drop('number', inplace=True, axis =1)
trainingdata.drop('Pin', inplace=True, axis =1)
trainingdata.drop('Street Address', inplace=True, axis =1)
trainingdata.drop('Amount Spent on Goods', inplace=True, axis =1)
trainingdata.drop('Retail Purchases – Most Frequent Category', inplace=True, axis =1)
trainingdata.drop('Estimated Income', inplace=True, axis =1)
amountava = updated['Amount Spent on Goods']
trainingdata['Gender'] = trainingdata.Gender.astype('category')
trainingdata['Method of Payment'] = trainingdata['Method of Payment'].astype('category')
trainingdata['Ville Address'] = trainingdata['Ville Address'].astype('category')
trainingdata['purchase'] = trainingdata['purchase'].astype('category')


le1= LabelEncoder()
le1.fit(trainingdata.Gender)
le2= LabelEncoder()
le2.fit(trainingdata.purchase)
le3= LabelEncoder()
le3.fit(trainingdata['Method of Payment'])
le4= LabelEncoder()
le4.fit(trainingdata['Ville Address'])
trainingdata1 = trainingdata.copy()
trainingdata1.Gender = le1.transform(trainingdata1.Gender)
trainingdata1.purchase = le2.transform(trainingdata1.purchase)
trainingdata1['Method of Payment'] = le3.transform(trainingdata1['Method of Payment'])
trainingdata1['Ville Address'] = le4.transform(trainingdata1['Ville Address'])
X_train, X_test, y_train, y_test = train_test_split(trainingdata1, amountava, test_size=0.33, random_state=42)

#clf = LinearRegression()
#clf.fit(X_train, y_train)

#clfd = DecisionTreeRegressor()
#

In [43]:
#clfd.fit(X_train, y_train)




trainingdata = updated2.copy()
incomemissing = updated2[updated2['Estimated Income'].isnull()].copy()
trainingdata.dropna(inplace=True)
incomesmiss = trainingdata['Estimated Income'].copy()
trainingdata.drop('number', inplace=True, axis =1)
trainingdata.drop('Pin', inplace=True, axis =1)
trainingdata.drop('Street Address', inplace=True, axis =1)
trainingdata.drop('Retail Purchases – Most Frequent Category', inplace=True, axis =1)
trainingdata.drop('Estimated Income', inplace=True, axis =1)
trainingdata['Gender'] = trainingdata.Gender.astype('category')
trainingdata['Method of Payment'] = trainingdata['Method of Payment'].astype('category')
trainingdata['Ville Address'] = trainingdata['Ville Address'].astype('category')
trainingdata['purchase'] = trainingdata['purchase'].astype('category')
le9= LabelEncoder()
le9.fit(trainingdata.Gender)
le2= LabelEncoder()
le2.fit(trainingdata.purchase)
le3= LabelEncoder()
le3.fit(trainingdata['Method of Payment'])
le4= LabelEncoder()
le4.fit(trainingdata['Ville Address'])
trainingdata1 = trainingdata.copy()
